In [46]:
import pandas as pd
from groq import Groq



In [6]:
import requests
import json
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta
import urllib3
urllib3.disable_warnings()

def get_ercot_data():
    """
    Gets ERCOT data following the Friday logic:
    1) If today is not Friday: 
       - Current = most recent Friday
       - Previous = Friday that is 3 weeks before the current Friday
    2) If today is Friday and can't get today's data:
       - Current = previous Friday 
       - Previous = Friday that is 3 weeks before that
    
    Returns:
        dict: Contains current_data, previous_data and their operating dates
    """
    # Login function
    def login(email="anwar@truelightenergy.com", password="anwar@truelightenergy.com"):
        url = "https://truepriceenergy.com/login"
        response = requests.post(url, params={"email": email, "password": password}, verify=False)
        return eval(response.text)["access_token"]
    
    # Get the most recent Friday
    def get_latest_friday(input_date):
        offset = (input_date.weekday() - 4) % 7
        return input_date - timedelta(days=offset)
    
    # Get data from API
    def fetch_data(operating_date):
        # Convert to date object if string
        if isinstance(operating_date, str):
            operating_date = datetime.strptime(operating_date, "%Y-%m-%d").date()
            
        op_date_str = operating_date.strftime("%Y-%m-%d")
        
        # Per your specifications:
        # Start date: First day of the month after operating month, same year
        operating_month = operating_date.month
        next_month = operating_month + 1
        year = operating_date.year
        
        if next_month > 12:
            next_month = 1
            year += 1
            
        start_date = date(year, next_month, 1).strftime("%Y-%m-%d")
        
        # End date: First day of the month after operating month, but in 2030
        end_date = date(2030, next_month, 1).strftime("%Y-%m-%d")
        
        # Make API request
        url = "https://truepriceenergy.com/get_data"
        query = {
            "start": start_date,
            "end": end_date,
            "operating_day": op_date_str,
            "curve_type": "nonenergy",
            "iso": "ercot",
            "strip": "standardized",
            "history": False,
            "type": "json"
        }
        
        token = login()
        headers = {"Authorization": f"Bearer {token}"}
        
        try:
            print(f"Fetching ERCOT data for operating date {op_date_str}")
            print(f"Start date: {start_date}, End date: {end_date}")
            
            response = requests.get(url, params=query, headers=headers, verify=False)
            data = json.loads(response.text)
            return data
        except Exception as e:
            print(f"Error fetching data for {op_date_str}: {str(e)}")
            return None
    
    # Determine operating dates based on current date
    today = date.today()
    is_friday = today.weekday() == 4
    
    # Get the most recent Friday
    latest_friday = get_latest_friday(today)
    
    # Set current and previous operating dates based on the logic
    if is_friday:
        # Today is Friday - check if we can get today's data
        try:
            current_data_test = fetch_data(today)
            if current_data_test:
                # Successfully got today's data
                current_op_date = today
                # Previous is 3 weeks before today
                previous_op_date = today - timedelta(weeks=3)
            else:
                # Can't get today's data, fall back to previous Friday
                current_op_date = latest_friday  # last Friday (should be same as today)
                # Previous is 3 weeks before current
                previous_op_date = current_op_date - timedelta(weeks=3)
        except:
            # Exception when fetching today's data
            current_op_date = latest_friday - timedelta(weeks=1)  # previous Friday
            # Previous is 3 weeks before current
            previous_op_date = current_op_date - timedelta(weeks=3)
    else:
        # Not Friday, use most recent Friday
        current_op_date = latest_friday 
        # Previous is 3 weeks before current
        previous_op_date = current_op_date - timedelta(weeks=4)
    
    # Fetch data for the determined dates
    current_data = fetch_data(current_op_date) 
    previous_data = fetch_data(previous_op_date)
    
    # Save to files
    if current_data:
        with open('current_ercot_data.json', 'w') as f:
            json.dump(current_data, f, indent=2)
        print(f"Saved current data (op date: {current_op_date.strftime('%Y-%m-%d')}) to current_ercot_data.json")
    else:
        print(f"Failed to retrieve or save current data for {current_op_date.strftime('%Y-%m-%d')}")
    
    if previous_data:
        with open('previous_ercot_data.json', 'w') as f:
            json.dump(previous_data, f, indent=2)
        print(f"Saved previous data (op date: {previous_op_date.strftime('%Y-%m-%d')}) to previous_ercot_data.json")
    else:
        print(f"Failed to retrieve or save previous data for {previous_op_date.strftime('%Y-%m-%d')}")
    
    return {
        "current_data": current_data,
        "previous_data": previous_data,
        "current_op_date": current_op_date.strftime("%Y-%m-%d"),
        "previous_op_date": previous_op_date.strftime("%Y-%m-%d")
    }

if __name__ == "__main__":
    result = get_ercot_data()
    print("\nERCOT Data Summary:")
    print(f"Current operating date: {result['current_op_date']}")
    print(f"Previous operating date: {result['previous_op_date']}")
    print(f"Difference in weeks: {(datetime.strptime(result['current_op_date'], '%Y-%m-%d').date() - datetime.strptime(result['previous_op_date'], '%Y-%m-%d').date()).days / 7}")
    
    if result['current_data']:
        print("Current data retrieved successfully")
    else:
        print("Failed to retrieve current data")
        
    if result['previous_data']:
        print("Previous data retrieved successfully")
    else:
        print("Failed to retrieve previous data")

Fetching ERCOT data for operating date 2025-05-09
Start date: 2025-06-01, End date: 2030-06-01
Fetching ERCOT data for operating date 2025-04-11
Start date: 2025-05-01, End date: 2030-05-01
Saved current data (op date: 2025-05-09) to current_ercot_data.json
Saved previous data (op date: 2025-04-11) to previous_ercot_data.json

ERCOT Data Summary:
Current operating date: 2025-05-09
Previous operating date: 2025-04-11
Difference in weeks: 4.0
Current data retrieved successfully
Previous data retrieved successfully


In [ ]:
result['previous_data'],result['current_data']


[{'Curve Start Month': '2026-01-01 00:00:00',
  'Data': 0.0,
  'Curve Update Date': '2025-04-11 12:00:00',
  'Control Area': 'ERCOT',
  'State': 'TX',
  'Load Zone': 'ALL',
  'Capacity Zone': 'ALL',
  'Utility': 'ALL',
  'Block Type': '7x24',
  'Cost Group': 'Ancillary Charges',
  'Cost Component': 'RMR - Capacity'},
 {'Curve Start Month': '2026-01-01 00:00:00',
  'Data': 7.0,
  'Curve Update Date': '2025-04-11 12:00:00',
  'Control Area': 'ERCOT',
  'State': 'TX',
  'Load Zone': 'ALL',
  'Capacity Zone': 'ALL',
  'Utility': 'ALL',
  'Block Type': '7x24',
  'Cost Group': 'Ancillary Charges - 4 Main',
  'Cost Component': 'NSRS'},
 {'Curve Start Month': '2026-01-01 00:00:00',
  'Data': -0.50812,
  'Curve Update Date': '2025-04-11 12:00:00',
  'Control Area': 'ERCOT',
  'State': 'TX',
  'Load Zone': 'ALL',
  'Capacity Zone': 'ALL',
  'Utility': 'ALL',
  'Block Type': '7x24',
  'Cost Group': 'Ancillary Charges',
  'Cost Component': 'Balancing Account Proceeds'},
 {'Curve Start Month': '202

In [27]:
def pivot_ercot_json(json_data):
    """
    Convert ERCOT JSON data into a pivoted DataFrame:
    Rows: Curve Start Month
    Columns: Cost Component
    Values: Data
    
    Args:
        json_data (list): List of dicts from the ERCOT JSON structure.

    Returns:
        pd.DataFrame: Pivoted DataFrame.
    """
    if not json_data:
        raise ValueError("Empty ERCOT JSON data")

    df = pd.DataFrame(json_data)

    # Ensure datetime format for sorting
    df['Curve Start Month'] = pd.to_datetime(df['Curve Start Month'])
    df['Curve Update Date'] = pd.to_datetime(df['Curve Update Date'])

    # Pivot table
    pivot_df = df.pivot_table(
        index='Curve Start Month',
        columns='Cost Component',
        values='Data',
        aggfunc='first'  # or 'sum'/'mean' if duplicates exist
    ).reset_index()

    # Optional: Add update date (assuming it's the same for each row)
    if 'Curve Update Date' in df.columns:
        pivot_df['Curve Update Date'] = df['Curve Update Date'].iloc[0]

    # Reorder columns (optional)
    cols = ['Curve Start Month', 'Curve Update Date'] + sorted([col for col in pivot_df.columns if col not in ['Curve Start Month', 'Curve Update Date']])
    return pivot_df[cols]


In [31]:
df2 = pivot_ercot_json(result['current_data'])
df1 = pivot_ercot_json(result['previous_data'])


In [33]:
df1['Curve Start Month'] = pd.to_datetime(df1['Curve Start Month'], utc=True)
df2['Curve Start Month'] = pd.to_datetime(df2['Curve Start Month'], utc=True)

# Step 2: Convert 'Curve Start Month' to UTC datetime
df1['Curve Start Month'] = pd.to_datetime(df1['Curve Start Month'], utc=True)
df2['Curve Start Month'] = pd.to_datetime(df2['Curve Start Month'], utc=True)

# Step 3: Identify cost columns (excluding the first two)
cost_columns = df1.columns[2:]


In [34]:
for col in cost_columns:
    df1[col] = pd.to_numeric(df1[col], errors='coerce')
    df2[col] = pd.to_numeric(df2[col], errors='coerce')

# Step 5: Align indices by shifting NEERCOT-1 forward one month
df1.set_index('Curve Start Month', inplace=True)
df2.set_index('Curve Start Month', inplace=True)
df1.index = df1.index + pd.DateOffset(months=1)

# Step 6: Calculate differences
df_diff = df2[cost_columns] - df1[cost_columns]
df_pct = ((df_diff / df1[cost_columns]) * 100).round(2)

# Step 7: Reset index for filtering
df_diff.reset_index(inplace=True)
df_pct.reset_index(inplace=True)


In [38]:
def filter_timeframe(df, start_date, end_date):
    mask = (df['Curve Start Month'] >= start_date) & (df['Curve Start Month'] <= end_date)
    return df.loc[mask]

In [39]:
from datetime import datetime, timedelta, timezone
from dateutil.relativedelta import relativedelta

today = datetime.now(timezone.utc)
first_of_next_month = datetime(today.year, today.month, 1, tzinfo=timezone.utc) + relativedelta(months=1)
last_of_next_month = first_of_next_month + relativedelta(day=31)

prompt_month = filter_timeframe(df_diff, first_of_next_month, last_of_next_month)
q2 = filter_timeframe(df_diff, datetime(2025, 4, 1, tzinfo=timezone.utc), datetime(2025, 6, 30, tzinfo=timezone.utc))
h1 = filter_timeframe(df_diff, datetime(2025, 1, 1, tzinfo=timezone.utc), datetime(2025, 6, 30, tzinfo=timezone.utc))
y2025 = filter_timeframe(df_diff, datetime(2025, 1, 1, tzinfo=timezone.utc), datetime(2025, 12, 31, tzinfo=timezone.utc))
summer = filter_timeframe(df_diff, datetime(2025, 6, 1, tzinfo=timezone.utc), datetime(2025, 9, 30, tzinfo=timezone.utc))


In [47]:

groq_api_key = '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'

groq_client = Groq(api_key=groq_api_key)


In [48]:

def get_groq_summary(prompt):
    response = groq_client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
    )
    return response.choices[0].message.content.strip()
def summarize(diff_df, pct_df, label):
    merged = diff_df.copy()
    for col in diff_df.columns[1:]:
        merged[col + " (%)"] = pct_df.set_index("Curve Start Month").loc[merged["Curve Start Month"], col].values
    # prompt = f"""
    # Act as an energy market editor. Summarize this {label} forecast in one high-level sentence combining trends:
    # - Market direction
    # - Notable cost category changes
    # - Concise tone

    # Data:
    # {merged.round(2).to_string(index=False)}
    # """
    prompt = f"""
    You are an expert energy analyst. Write **3 crisp bullet points** summarizing trends from the following data. **Do not add any introductions** like "Here is the summary" — only the 3 points.

    Focus on:
    - Market direction (up/down/mixed)
    - Big increases/decreases in categories
    - Keep it professional, direct, and data-driven

    Data:
    {merged.round(2).to_string(index=False)}
    """


    return get_groq_summary(prompt)

In [49]:
summaries = {
    "Prompt Month": summarize(prompt_month, df_pct, "Prompt Month"),
    "Q2": summarize(q2, df_pct, "Q2"),
    "H1": summarize(h1, df_pct, "H1"),
    "Year 2025": summarize(y2025, df_pct, "Year 2025"),
    "Summer": summarize(summer, df_pct, "Summer")
}



In [50]:
summaries

{'Prompt Month': '• The overall market trend is neutral, as most categories show minimal changes, with only a few notable exceptions.\n• Notable increases are seen in ECRS increase of 2.98%, RUC increase of 3.83%, and VSS - Lost Opportunity increase of 57.5%.\n• Significant decreases are observed in Emergency Energy (-47.5%), RMR - Capacity (-37.14%), and VSS - Reactive Power (-11.67%).',
 'Q2': '• The overall market trend is downward, driven by decreases in RMR - Capacity (-47.5%) and RMR - Services (-37.14%).\n• There are significant increases in Ancillary Service Settlement (25.0%) and ORDC Settlement (2.19%), indicating a shift towards these services.\n• The largest percentage decreases are in Emergency Energy (-57.5%) and VSS - Lost Opportunity (-47.5%), suggesting a decline in these areas.',
 'H1': '• The overall market trend is neutral, as most categories show minimal changes, with only a few notable exceptions.\n• The biggest decreases are seen in RMR - Capacity (-47.5%) and Em

In [1]:
import requests
import json
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
import urllib3

urllib3.disable_warnings()

def login(email="anwar@truelightenergy.com", password="anwar@truelightenergy.com"):
    url = "https://truepriceenergy.com/login"
    response = requests.post(url, params={"email": email, "password": password}, verify=False)
    return eval(response.text)["access_token"]

def get_last_day_of_month(d):
    return (d.replace(day=1) + relativedelta(months=1)) - timedelta(days=1)

def fetch_data(operating_date):
    if isinstance(operating_date, str):
        operating_date = datetime.strptime(operating_date, "%Y-%m-%d").date()
        
    op_date_str = operating_date.strftime("%Y-%m-%d")

    # Start = First day of month after op_date
    next_month = operating_date + relativedelta(months=1)
    start_date = date(next_month.year, next_month.month, 1).strftime("%Y-%m-%d")
    end_date = date(2030, next_month.month, 1).strftime("%Y-%m-%d")

    url = "https://truepriceenergy.com/get_data"
    query = {
        "start": start_date,
        "end": end_date,
        "operating_day": op_date_str,
        "curve_type": "nonenergy",
        "iso": "isone",
        "strip": "standardized",
        "history": False,
        "type": "json"
    }

    token = login()
    headers = {"Authorization": f"Bearer {token}"}
    try:
        print(f"Fetching ISONE data for operating date {op_date_str}")
        print(f"Start date: {start_date}, End date: {end_date}")
        response = requests.get(url, params=query, headers=headers, verify=False)
        return json.loads(response.text)
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

def get_isone_data():
    today = date.today()
    last_day_this_month = get_last_day_of_month(today)

    if today == last_day_this_month:
        current_op_date = today
        previous_op_date = get_last_day_of_month(today - relativedelta(months=1))
    else:
        current_op_date = get_last_day_of_month(today - relativedelta(months=1))
        previous_op_date = get_last_day_of_month(today - relativedelta(months=2))

    current_data = fetch_data(current_op_date)
    previous_data = fetch_data(previous_op_date)

    # Save to files (optional)
    if current_data:
        with open('current_isone_data.json', 'w') as f:
            json.dump(current_data, f, indent=2)
        print(f"Saved current data (op date: {current_op_date}) to current_isone_data.json")
    else:
        print(f"Failed to retrieve or save current data for {current_op_date}")

    # if previous_data:
    #     with open('previous_isone_data.json', 'w') as f:
    #         json.dump(previous_data, f, indent=2)
    #     print(f"Saved previous data (op date: {previous_op_date}) to previous_isone_data.json")
    # else:
    #     print(f"Failed to retrieve or save previous data for {previous_op_date}")

    return {
        "current_data": current_data,
        "previous_data": previous_data,
        "current_op_date": current_op_date.strftime("%Y-%m-%d"),
        "previous_op_date": previous_op_date.strftime("%Y-%m-%d")
    }

if __name__ == "__main__":
    result = get_isone_data()
    print("\nISONE Data Summary:")
    print(f"Current operating date: {result['current_op_date']}")
    print(f"Previous operating date: {result['previous_op_date']}")
    print(f"Difference in days: {(datetime.strptime(result['current_op_date'], '%Y-%m-%d').date() - datetime.strptime(result['previous_op_date'], '%Y-%m-%d').date()).days}")
    
    if result['current_data']:
        print("✅ Current data retrieved successfully")
    else:
        print("❌ Failed to retrieve current data")

    if result['previous_data']:
        print("✅ Previous data retrieved successfully")
    else:
        print("❌ Failed to retrieve previous data")


Fetching ISONE data for operating date 2025-04-30
Start date: 2025-05-01, End date: 2030-05-01
Fetching ISONE data for operating date 2025-03-31
Start date: 2025-04-01, End date: 2030-04-01
Saved current data (op date: 2025-04-30) to current_isone_data.json

ISONE Data Summary:
Current operating date: 2025-04-30
Previous operating date: 2025-03-31
Difference in days: 30
✅ Current data retrieved successfully
✅ Previous data retrieved successfully


In [3]:
result['previous_data']

[{'Curve Start Month': '2028-01-01 00:00:00',
  'Data': 0.17779,
  'Curve Update Date': '2025-03-31 12:00:00',
  'Control Area': 'ISONE',
  'State': 'VT',
  'Load Zone': 'VERMONT',
  'Capacity Zone': 'ALL',
  'Utility': 'ALL',
  'Block Type': '7x24',
  'Cost Group': 'NCPC',
  'Cost Component': 'Second Contingency'},
 {'Curve Start Month': '2028-01-01 00:00:00',
  'Data': 0.00039,
  'Curve Update Date': '2025-03-31 12:00:00',
  'Control Area': 'ISONE',
  'State': 'CT',
  'Load Zone': 'CONNECTICUT',
  'Capacity Zone': 'ALL',
  'Utility': 'ALL',
  'Block Type': '7x24',
  'Cost Group': 'NCPC',
  'Cost Component': 'RT Economic LSCPR'},
 {'Curve Start Month': '2028-01-01 00:00:00',
  'Data': 0.19166,
  'Curve Update Date': '2025-03-31 12:00:00',
  'Control Area': 'ISONE',
  'State': 'ALL',
  'Load Zone': 'ALL',
  'Capacity Zone': 'ALL',
  'Utility': 'ALL',
  'Block Type': '7x24',
  'Cost Group': 'ANCILLARIES',
  'Cost Component': 'NEPOOL Exp'},
 {'Curve Start Month': '2028-01-01 00:00:00',
 